In [1]:
import numpy as np
import pandas as pd

In [2]:
import numpy as np
import pandas as pd
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [4]:
Data1=pd.read_csv(r"C:\Users\Saurabh\Desktop\Aires1\processed_data2.csv")

In [5]:
Data1.drop(["Unnamed: 0"],axis=1,inplace=True)

In [6]:
Data1.columns

Index(['company_id', 'corporation_policy_id', 'assignment_id',
       'assignment_type_code', 'family_size', 'CITY_ORIG', 'COUNTRY_CODE_ORIG',
       'CITY_DEST', 'COUNTRY_CODE_DEST', 'home_status_code',
       'assign_sub_service_code', 'sub_service_status_code', 'assign_duration',
       'amount'],
      dtype='object')

In [7]:
Data1["assign_duration"]=Data1["assign_duration"].str.replace(" days 00:00:00","")
Data1["assign_duration"]=Data1["assign_duration"].str.replace(" days","")
Data1["assign_duration"]=Data1["assign_duration"].str.split(" ")
Data1["assign_duration"]=Data1["assign_duration"].str[0]
Data1["assign_duration"]=Data1["assign_duration"].astype(int)

In [76]:
Data1.shape

(42858, 14)

In [9]:
Data_Val1  = Data1[Data1["assignment_id"].isin([463279,509353,501516,480737,517941,481203,483108,521227,
                                                             491086,532478,474347,482464,597107,581689,609902,467567])]

In [77]:
Data_Val1.shape

(90, 14)

In [11]:
Data_numerical_features = Data1.select_dtypes(exclude='object')
Data_categorical_features = Data1.select_dtypes(include='object')

In [12]:
def outlier_cap(x):
    x=x.clip(lower=x.quantile(0.05))
    x=x.clip(upper=x.quantile(0.95))
    return(x)

In [13]:
num_new=Data_numerical_features.iloc[:,1:].apply(lambda x : outlier_cap(x))

In [14]:
Data2=pd.concat([num_new,Data_categorical_features],axis=1)

In [15]:
Data2

,corporation_policy_id,assignment_id,family_size,assign_duration,amount,assignment_type_code,CITY_ORIG,COUNTRY_CODE_ORIG,CITY_DEST,COUNTRY_CODE_DEST,home_status_code,assign_sub_service_code,sub_service_status_code
0,9428.0,467387.0,2.0,62.0,2947.532222,PERM,melbourne,US,palmdale,US,OWN,SHIPMENT,INAC
1,9428.0,480183.0,4.0,270.0,2032.007692,PERM,kildeer,US,clifton,US,OWN,HOMEPRCH,INAC
2,9428.0,480183.0,4.0,270.0,4061.642976,PERM,kildeer,US,clifton,US,OWN,HOMESALE,INAC
3,9428.0,480183.0,4.0,270.0,3127.551667,PERM,kildeer,US,clifton,US,OWN,SHIPMENT,INAC
4,9428.0,481230.0,4.0,88.0,1673.444444,PERM,fulton,US,san diego,US,OWN,HOMEPRCH,INAC
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42853,9436.0,504685.0,1.0,0.0,200.000000,PERM,santa ana,US,san diego,US,RENT,AIRESFEE_AIAI,INAC
42854,9429.0,474678.0,1.0,0.0,650.000000,PERM,reston,US,odgen,US,RENT,AIRESFEE_AIAI,INAC
42855,9436.0,519543.0,1.0,0.0,200.000000,PERM,madison,US,rolling meadows,US,RENT,AIRESFEE_AIAI,INAC
42856,9436.0,468302.0,1.0,0.0,200.000000,PERM,san diego,US,redondo beach,US,RENT,AIRESFEE_AIAI,INAC


In [16]:
Data2=pd.concat([Data_numerical_features["company_id"],Data2],axis=1)

In [17]:
Data2

,company_id,corporation_policy_id,assignment_id,family_size,assign_duration,amount,assignment_type_code,CITY_ORIG,COUNTRY_CODE_ORIG,CITY_DEST,COUNTRY_CODE_DEST,home_status_code,assign_sub_service_code,sub_service_status_code
0,78223,9428.0,467387.0,2.0,62.0,2947.532222,PERM,melbourne,US,palmdale,US,OWN,SHIPMENT,INAC
1,78223,9428.0,480183.0,4.0,270.0,2032.007692,PERM,kildeer,US,clifton,US,OWN,HOMEPRCH,INAC
2,78223,9428.0,480183.0,4.0,270.0,4061.642976,PERM,kildeer,US,clifton,US,OWN,HOMESALE,INAC
3,78223,9428.0,480183.0,4.0,270.0,3127.551667,PERM,kildeer,US,clifton,US,OWN,SHIPMENT,INAC
4,78223,9428.0,481230.0,4.0,88.0,1673.444444,PERM,fulton,US,san diego,US,OWN,HOMEPRCH,INAC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42853,78223,9436.0,504685.0,1.0,0.0,200.000000,PERM,santa ana,US,san diego,US,RENT,AIRESFEE_AIAI,INAC
42854,78223,9429.0,474678.0,1.0,0.0,650.000000,PERM,reston,US,odgen,US,RENT,AIRESFEE_AIAI,INAC
42855,78223,9436.0,519543.0,1.0,0.0,200.000000,PERM,madison,US,rolling meadows,US,RENT,AIRESFEE_AIAI,INAC
42856,78223,9436.0,468302.0,1.0,0.0,200.000000,PERM,san diego,US,redondo beach,US,RENT,AIRESFEE_AIAI,INAC


In [18]:
encoder = TargetEncoder()
Data2['CITY_ORIG'] = encoder.fit_transform(Data2['CITY_ORIG'], Data2['amount'])

C:\Users\Saurabh\anaconda3\envs\Claimfraud\lib\site-packages\category_encoders\target_encoder.py:92: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
C:\Users\Saurabh\anaconda3\envs\Claimfraud\lib\site-packages\category_encoders\target_encoder.py:97: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [19]:
Data2["COUNTRY_CODE_ORIG"]=encoder.fit_transform(Data2['COUNTRY_CODE_ORIG'], Data2['amount'])

In [20]:
Data2["COUNTRY_CODE_DEST"]=encoder.fit_transform(Data2['COUNTRY_CODE_DEST'], Data2['amount'])

In [21]:
Data2['CITY_DEST'] = encoder.fit_transform(Data2['CITY_DEST'], Data2['amount'])

In [22]:
Data_main=pd.get_dummies(data=Data2, columns=['assignment_type_code','home_status_code','assign_sub_service_code','sub_service_status_code'])

In [25]:
Data_main = Data_main.loc[:,~Data_main.columns.duplicated()].copy()

In [23]:
Data_main

,company_id,corporation_policy_id,assignment_id,family_size,assign_duration,amount,CITY_ORIG,COUNTRY_CODE_ORIG,CITY_DEST,COUNTRY_CODE_DEST,...,assign_sub_service_code_TRVLASST,assign_sub_service_code_TRVLFTRP,assign_sub_service_code_TRVLHMLV,assign_sub_service_code_TRVLHMSR,assign_sub_service_code_TRVLPREA,assign_sub_service_code_TRVLSHRT,sub_service_status_code_ACTV,sub_service_status_code_CNCL,sub_service_status_code_INAC,sub_service_status_code_XCST
0,78223,9428.0,467387.0,2.0,62.0,2947.532222,1915.596195,2526.523571,1366.308550,2550.462195,...,0,0,0,0,0,0,0,0,1,0
1,78223,9428.0,480183.0,4.0,270.0,2032.007692,2714.077953,2526.523571,2007.062027,2550.462195,...,0,0,0,0,0,0,0,0,1,0
2,78223,9428.0,480183.0,4.0,270.0,4061.642976,2714.077953,2526.523571,2007.062027,2550.462195,...,0,0,0,0,0,0,0,0,1,0
3,78223,9428.0,480183.0,4.0,270.0,3127.551667,2714.077953,2526.523571,2007.062027,2550.462195,...,0,0,0,0,0,0,0,0,1,0
4,78223,9428.0,481230.0,4.0,88.0,1673.444444,3185.671143,2526.523571,1667.877001,2550.462195,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42853,78223,9436.0,504685.0,1.0,0.0,200.000000,2738.422000,2526.523571,1667.877001,2550.462195,...,0,0,0,0,0,0,0,0,1,0
42854,78223,9429.0,474678.0,1.0,0.0,650.000000,2190.704456,2526.523571,967.104342,2550.462195,...,0,0,0,0,0,0,0,0,1,0
42855,78223,9436.0,519543.0,1.0,0.0,200.000000,2927.829656,2526.523571,2182.896763,2550.462195,...,0,0,0,0,0,0,0,0,1,0
42856,78223,9436.0,468302.0,1.0,0.0,200.000000,1730.274315,2526.523571,1387.541424,2550.462195,...,0,0,0,0,0,0,0,0,1,0


In [24]:
Data_main["company_id"]=Data_main["company_id"].map({78223:"A",60915:"B",8472:"C",45474:"D"})

In [25]:
Data_main=pd.get_dummies(data=Data_main, columns=['company_id'])

In [26]:
Data_main

,corporation_policy_id,assignment_id,family_size,assign_duration,amount,CITY_ORIG,COUNTRY_CODE_ORIG,CITY_DEST,COUNTRY_CODE_DEST,assignment_type_code_CMTR,...,assign_sub_service_code_TRVLPREA,assign_sub_service_code_TRVLSHRT,sub_service_status_code_ACTV,sub_service_status_code_CNCL,sub_service_status_code_INAC,sub_service_status_code_XCST,company_id_A,company_id_B,company_id_C,company_id_D
0,9428.0,467387.0,2.0,62.0,2947.532222,1915.596195,2526.523571,1366.308550,2550.462195,0,...,0,0,0,0,1,0,1,0,0,0
1,9428.0,480183.0,4.0,270.0,2032.007692,2714.077953,2526.523571,2007.062027,2550.462195,0,...,0,0,0,0,1,0,1,0,0,0
2,9428.0,480183.0,4.0,270.0,4061.642976,2714.077953,2526.523571,2007.062027,2550.462195,0,...,0,0,0,0,1,0,1,0,0,0
3,9428.0,480183.0,4.0,270.0,3127.551667,2714.077953,2526.523571,2007.062027,2550.462195,0,...,0,0,0,0,1,0,1,0,0,0
4,9428.0,481230.0,4.0,88.0,1673.444444,3185.671143,2526.523571,1667.877001,2550.462195,0,...,0,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42853,9436.0,504685.0,1.0,0.0,200.000000,2738.422000,2526.523571,1667.877001,2550.462195,0,...,0,0,0,0,1,0,1,0,0,0
42854,9429.0,474678.0,1.0,0.0,650.000000,2190.704456,2526.523571,967.104342,2550.462195,0,...,0,0,0,0,1,0,1,0,0,0
42855,9436.0,519543.0,1.0,0.0,200.000000,2927.829656,2526.523571,2182.896763,2550.462195,0,...,0,0,0,0,1,0,1,0,0,0
42856,9436.0,468302.0,1.0,0.0,200.000000,1730.274315,2526.523571,1387.541424,2550.462195,0,...,0,0,0,0,1,0,1,0,0,0


In [27]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()
Data_main['corporation_policy_id'] = le.fit_transform(Data_main['corporation_policy_id'])

In [28]:
Data_Val  = Data_main[Data_main["assignment_id"].isin([463279,509353,501516,480737,517941,481203,483108,521227,
                                                             491086,532478,474347,482464,597107,581689,609902,467567])]

In [78]:
Data_Val1  = Data1[Data1["assignment_id"].isin([463279,509353,501516,480737,517941,481203,483108,521227,
                                                             491086,532478,474347,482464,597107,581689,609902,467567])]

In [79]:
Data_Val1.shape

(90, 14)

In [29]:
Data_Val

,corporation_policy_id,assignment_id,family_size,assign_duration,amount,CITY_ORIG,COUNTRY_CODE_ORIG,CITY_DEST,COUNTRY_CODE_DEST,assignment_type_code_CMTR,...,assign_sub_service_code_TRVLPREA,assign_sub_service_code_TRVLSHRT,sub_service_status_code_ACTV,sub_service_status_code_CNCL,sub_service_status_code_INAC,sub_service_status_code_XCST,company_id_A,company_id_B,company_id_C,company_id_D
277,0,480737.0,1.0,136.0,749.933333,1915.596195,2526.523571,1644.144203,2550.462195,0,...,0,0,0,0,1,0,1,0,0,0
278,0,480737.0,1.0,136.0,4335.118438,1915.596195,2526.523571,1644.144203,2550.462195,0,...,0,0,0,0,1,0,1,0,0,0
279,0,481203.0,1.0,141.0,10000.000000,2530.938479,2526.523571,1730.564092,2550.462195,0,...,0,0,0,0,1,0,1,0,0,0
280,0,481203.0,1.0,141.0,3210.317143,2530.938479,2526.523571,1730.564092,2550.462195,0,...,0,0,0,0,1,0,1,0,0,0
293,0,483108.0,2.0,56.0,903.816364,1939.403669,2526.523571,2264.557320,2550.462195,0,...,0,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30702,70,532478.0,1.0,77.0,1980.000000,2034.939107,2526.523571,2302.102324,2550.462195,0,...,0,0,0,0,1,0,0,1,0,0
32895,0,483108.0,2.0,56.0,350.000000,1939.403669,2526.523571,2264.557320,2550.462195,0,...,0,0,0,0,1,0,1,0,0,0
37854,0,481203.0,1.0,141.0,4902.000000,2530.938479,2526.523571,1730.564092,2550.462195,0,...,0,0,0,0,1,0,1,0,0,0
37855,0,481203.0,1.0,141.0,350.000000,2530.938479,2526.523571,1730.564092,2550.462195,0,...,0,0,0,0,1,0,1,0,0,0


In [30]:
Data_Val_index  = Data_main[Data_main["assignment_id"].isin([463279,509353,501516,480737,517941,481203,483108,521227,
                                                             491086,532478,474347,482464,597107,581689,609902,467567])].index

In [31]:
Data_main.drop(Data_Val_index, inplace = True)

In [32]:
Data_main.drop(["assignment_id"],axis=1,inplace=True)

In [33]:
Data_Val.drop(["assignment_id"],axis=1,inplace=True)

C:\Users\Saurabh\AppData\Local\Temp\ipykernel_13984\1944881090.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data_Val.drop(["assignment_id"],axis=1,inplace=True)


In [34]:

# Create correlation matrix
corr_matrix = Data_main.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.80)]

# Drop features 
#df.drop(to_drop, axis=1, inplace=True)

C:\Users\Saurabh\AppData\Local\Temp\ipykernel_13984\4044336726.py:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


In [35]:
to_drop

['sub_service_status_code_INAC', 'company_id_A']

In [36]:
Data_main.drop(to_drop,axis=1,inplace=True)

In [37]:
Data_Val.drop(to_drop,axis=1,inplace=True)

C:\Users\Saurabh\AppData\Local\Temp\ipykernel_13984\3853751545.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data_Val.drop(to_drop,axis=1,inplace=True)


In [38]:
amount_Val=Data_Val["amount"]

In [39]:
amount=Data_main["amount"]

In [52]:
amount

0        2947.532222
1        2032.007692
2        4061.642976
3        3127.551667
4        1673.444444
            ...     
42853     200.000000
42854     650.000000
42855     200.000000
42856     200.000000
42857     200.000000
Name: amount, Length: 42768, dtype: float64

In [40]:
Data_Val.drop(["amount"],axis=1,inplace=True)

C:\Users\Saurabh\AppData\Local\Temp\ipykernel_13984\1876593826.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data_Val.drop(["amount"],axis=1,inplace=True)


In [41]:
Data_main.drop(["amount"],axis=1,inplace=True)

In [66]:
Data_main.to_excel(r"C:\Users\Saurabh\Desktop\Aires1\abc.xlsx")

In [42]:
#X = Data_processed.drop(['actual_amount'],axis=1,inplace=True)
y = np.array(amount).reshape(-1, 1)
X= Data_main
# Separating the data into independent and dependent variables
# Dropping any rows with Nan values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

# Splitting the data into training and testing data
regr = LinearRegression()

regr.fit(X_train, y_train)
print(regr.score(X_test, y_test))


0.672129116005366


In [43]:
import xgboost as xg
from sklearn.metrics import mean_squared_error as MSE


# Instantiation
xgb_r = xg.XGBRegressor(objective ='reg:squarederror',n_estimators = 1000)

# Fitting the model
xgb_r.fit(X_train, y_train)

# Predict the model
pred = xgb_r.predict(X_test)

# RMSE Computation
rmse = np.sqrt(MSE(y_test, pred))
print("RMSE : % f" %(rmse))


RMSE :  1409.359608


In [44]:
print(xgb_r.score(X_test, y_test))

0.7608238835115354


In [45]:
from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_percentage_error(y_test, pred)

0.6199524958242848

In [46]:
X_train1=Data_main
y_train1=y

In [47]:
y_Val = np.array(amount_Val).reshape(-1, 1)
X_Val=Data_Val

In [50]:
import xgboost as xg
from sklearn.metrics import mean_squared_error as MSE


# Instantiation
xgb_r1 = xg.XGBRegressor(objective ='reg:squarederror',n_estimators = 1000)

# Fitting the model
xgb_r1.fit(X_train1, y_train1)

# Predict the model
pred1 = xgb_r1.predict(X_Val)

# RMSE Computation
rmse = np.sqrt(MSE(y_Val, pred1))
print("RMSE : % f" %(rmse))


RMSE :  1985.916189


In [51]:
print(xgb_r1.score(X_Val, y_Val))

0.4678110907666766


In [52]:
from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_percentage_error(y_Val, pred1)

0.7645019649197252

In [53]:
df=pd.DataFrame()

In [59]:
y1=y_Val.tolist()

In [61]:
flat_list = []
for sublist in y1:
    for item in sublist:
        flat_list.append(item)

In [63]:
df["amount"]=flat_list
df["predicted_amount"]=pred1.tolist()

In [73]:
df_merged = pd.concat([Data_Val.reset_index(drop=True),df.reset_index(drop=True)], axis=1)

In [74]:
df_merged

,corporation_policy_id,family_size,assign_duration,CITY_ORIG,COUNTRY_CODE_ORIG,CITY_DEST,COUNTRY_CODE_DEST,assignment_type_code_CMTR,assignment_type_code_INTR,assignment_type_code_LGTM,...,sub_service_status_code_CNCL,sub_service_status_code_XCST,company_id_B,company_id_C,company_id_D,amount,predicted_amount,diff,diff_abs,error
0,0,1.0,136.0,1915.596195,2526.523571,1644.144203,2550.462195,0,0,0,...,0,0,0,0,0,749.933333,884.401611,-134.468278,134.468278,17.930698
1,0,1.0,136.0,1915.596195,2526.523571,1644.144203,2550.462195,0,0,0,...,0,0,0,0,0,4335.118438,4794.763184,-459.644746,459.644746,10.602819
2,0,1.0,141.0,2530.938479,2526.523571,1730.564092,2550.462195,0,0,0,...,0,0,0,0,0,10000.000000,4601.334961,5398.665039,5398.665039,53.986650
3,0,1.0,141.0,2530.938479,2526.523571,1730.564092,2550.462195,0,0,0,...,0,0,0,0,0,3210.317143,3157.936523,52.380619,52.380619,1.631634
4,0,2.0,56.0,1939.403669,2526.523571,2264.557320,2550.462195,0,0,0,...,0,0,0,0,0,903.816364,2670.729248,-1766.912884,1766.912884,195.494677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,70,1.0,77.0,2034.939107,2526.523571,2302.102324,2550.462195,0,0,0,...,0,0,1,0,0,1980.000000,1605.953369,374.046631,374.046631,18.891244
77,0,2.0,56.0,1939.403669,2526.523571,2264.557320,2550.462195,0,0,0,...,0,0,0,0,0,350.000000,447.493225,-97.493225,97.493225,27.855207
78,0,1.0,141.0,2530.938479,2526.523571,1730.564092,2550.462195,0,0,0,...,0,0,0,0,0,4902.000000,1681.490845,3220.509155,3220.509155,65.697861
79,0,1.0,141.0,2530.938479,2526.523571,1730.564092,2550.462195,0,0,0,...,0,0,0,0,0,350.000000,258.504547,91.495453,91.495453,26.141558


In [69]:
df["diff"]=df["amount"]-df["predicted_amount"]

In [70]:
df["diff_abs"]=df["diff"].abs()

In [71]:
df["error"]=(df["diff_abs"]/df["amount"])*100

In [72]:
df

,amount,predicted_amount,diff,diff_abs,error
0,749.933333,884.401611,-134.468278,134.468278,17.930698
1,4335.118438,4794.763184,-459.644746,459.644746,10.602819
2,10000.000000,4601.334961,5398.665039,5398.665039,53.986650
3,3210.317143,3157.936523,52.380619,52.380619,1.631634
4,903.816364,2670.729248,-1766.912884,1766.912884,195.494677
...,...,...,...,...,...
76,1980.000000,1605.953369,374.046631,374.046631,18.891244
77,350.000000,447.493225,-97.493225,97.493225,27.855207
78,4902.000000,1681.490845,3220.509155,3220.509155,65.697861
79,350.000000,258.504547,91.495453,91.495453,26.141558


In [75]:
df_merged.to_excel(r"C:\Users\Saurabh\Desktop\Aires1\predicted_data.xlsx")

In [22]:
#XGBoost hyper-parameter tuning
from sklearn.model_selection import GridSearchCV
def hyperParameterTuningGCV(X_train, y_train):
    param_tuning = {
        'max_depth': [2,5,10],
        'learning_rate': [0.01,0.1,0.2],
        'n_estimators': [500,1000,1200],
        'colsample_bytree': [0.2,0.7],
        'booster':['gbtree'],
        'objective': ['reg:squarederror'] }
       ## Hyper Parameter Optimization
#     n_estimators = [100, 500, 900, 1100, 1500]
#     max_depth = [2, 3, 5, 10, 15]
#     booster=['gbtree','gblinear']
#     learning_rate=[0.05,0.1,0.15,0.20]
#     min_child_weight=[1,2,3,4]

    xgb_model = xg.XGBRegressor()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           #scoring = 'neg_mean_squared_error',  #MSE
                           cv = 5,verbose=1)
    gsearch.fit(X_train,y_train)

    return gsearch.best_params_,gsearch.best_estimator_,gsearch.score(X_test,y_test),gsearch

In [23]:
best_par,best_est,r2_score_gcv,model_gcv=hyperParameterTuningGCV(X_train,y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [24]:
r2_score_gcv

0.790955531662448

In [33]:
model_gcv

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    gamma=None, gpu_id=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bin=None,
                                    max_cat...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None, ...),
             param_grid={'booster': ['gbtree'], 'colsample_bytree': [0.7],
                         'learning_rate': [0.01], 'max_depth': [9],
                         'n_estimators': [1000],
                         'objective': ['reg:squarederror']},
             verbose=1)

In [25]:
best_par

{'booster': 'gbtree',
 'colsample_bytree': 0.7,
 'learning_rate': 0.01,
 'max_depth': 9,
 'n_estimators': 1000,
 'objective': 'reg:squarederror'}

In [26]:
feature_importance=model_gcv.best_estimator_.feature_importances_

In [27]:
feature_importance

array([0.00527293, 0.00090632, 0.00557621, 0.00566405, 0.00128873,
       0.00222227, 0.00854183, 0.00087024, 0.00078241, 0.01682754,
       0.0149306 , 0.01482656, 0.06290586, 0.01458749, 0.00275979,
       0.0087202 , 0.00212319, 0.0011575 , 0.00230181, 0.00468866,
       0.03820064, 0.00552279, 0.05210909, 0.61223763, 0.01505789,
       0.00580807, 0.00324677, 0.0007883 , 0.00274955, 0.07684098,
       0.00542964, 0.00505445], dtype=float32)

In [28]:
# Indices of the sorted elements of a
# given array
indices = np.argsort(feature_importance)
indices


array([ 8, 27,  7,  1, 17,  4, 16,  5, 18, 28, 14, 26, 19, 31,  0, 30, 21,
        2,  3, 25,  6, 15, 13, 11, 10, 24,  9, 20, 22, 12, 29, 23],
      dtype=int64)

In [29]:
pos = [25,  3, 21,  6, 15, 10, 24, 11, 13,  9, 20, 22, 12, 29, 23]
colname = Data1.columns[pos]

In [37]:
colname

Index(['assign_sub_service_code_RENTLASS', 'tl_num_of_bedroom',
       'assign_sub_service_code_HOMEPRCH', 'Freq_COUNTRY_CODE_DEST',
       'assignment_service_code_REALEST', 'company_id_C',
       'assign_sub_service_code_NASINVOI', 'company_id_D',
       'assignment_service_code_HOSTCTRY', 'company_id_B',
       'assign_sub_service_code_EXPNFORM', 'assign_sub_service_code_HOMESALE',
       'assignment_service_code_EXPENSE', 'assign_sub_service_code_others',
       'assign_sub_service_code_LUMPSUM'],
      dtype='object')

In [32]:
# select features using threshold
# Fit model using each importance as a threshold
from sklearn.feature_selection import SelectFromModel
import xgboost as xg
from sklearn.metrics import mean_squared_error as MSE
thresholds = np.sort(feature_importance)
for thresh in thresholds:
    selection = SelectFromModel(model_gcv, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_train)
    # train model
    selection_model = xg.XGBRegressor(objective ='reg:squarederror',n_estimators = 1000, seed = 123)
    selection_model.fit(select_X_train, y_train)
    # eval model
    select_X_test = selection.transform(X_test)
    y_pred = selection_model.predict(select_X_test)
    #predictions = [round(value) for value in y_pred]
    #accuracy = selection_model.score(y_test, predictions)
    rmse = np.sqrt(MSE(y_test, y_pred))
    #print("RMSE : % f" %(rmse))
    r2score=selection_model.score(select_X_test, y_test)
    print("Thresh=%.3f, n=%d, rmse=%.5f,r2score: %.2f%%" % (thresh, select_X_train.shape[1],rmse, r2score*100.0))

C:\Users\Saurabh\anaconda3\envs\Claimfraud\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


ValueError: when `importance_getter=='auto'`, the underlying estimator GridSearchCV should have `coef_` or `feature_importances_` attribute. Either pass a fitted estimator to feature selector or call fit before calling transform.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
def hyperParameterTuningRCV(X_train, y_train):

    ## Hyper Parameter Optimization
#     n_estimators = [100, 500, 900, 1100, 1500]
#     max_depth = [2, 3, 5, 10, 15]
#     booster=['gbtree','gblinear']
#     learning_rate=[0.05,0.1,0.15,0.20]
#     min_child_weight=[1,2,3,4]
    
    n_estimators = [900]
    max_depth = [3]
    booster=['gbtree']
    learning_rate=[0.20]
    min_child_weight=[2]
    objective=['reg:squarederror']

    # Define the grid of hyperparameters to search
    hyperparameter_grid = {
        'n_estimators': n_estimators,
        'max_depth':max_depth,
        'learning_rate':learning_rate,
        'min_child_weight':min_child_weight,
        'booster':booster,
        'objective': objective
        }
    xgb_model_r = xg.XGBRegressor()

    # Set up the random search with 4-fold cross validation
    random_cv = RandomizedSearchCV(estimator=xgb_model_r,
                param_distributions=hyperparameter_grid,
                cv=5, n_iter=25,
                verbose = 1, 
                return_train_score = True,
                random_state=42)

    random_cv.fit(X_train,y_train)

    return random_cv.best_estimator_,random_cv.score(X_test,y_test)


In [ ]:
best_estimator,r2_score=hyperParameterTuningRCV(X_train, y_train)

In [ ]:
r2_score

In [ ]:
best_estimator

In [ ]:
# { 'max_depth': [3, 5, 6, 10, 15, 20],
#            'learning_rate': [0.01, 0.1, 0.2, 0.3],
#            'subsample': np.arange(0.5, 1.0, 0.1),
#            'colsample_bytree': np.arange(0.4, 1.0, 0.1),
#            'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
#            'n_estimators': [100, 500, 1000]}

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# create ANN model
model = Sequential()

# Defining the Input layer and FIRST hidden layer, both are same!
model.add(Dense(units=30, input_dim=32, kernel_initializer='normal', activation='relu'))
#model.add(Dropout(0.2))

# Defining the Second layer of the model
# after the first layer we don't have to specify input_dim as keras configure it automatically
model.add(Dense(units=30, kernel_initializer='normal', activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(units=35, kernel_initializer='normal', activation='relu'))
# The output neuron is a single fully connected node 
# Since we will be predicting a single number
model.add(Dense(1, kernel_initializer='normal'))

# Compiling the model
model.compile(loss='mean_squared_error', optimizer='adam')
# Fitting the ANN to the Training set
model.fit(X_train, y_train ,batch_size = 10, epochs = 50, verbose=1)

# Generating Predictions on testing data
Predictions_orig=model.predict(X_test)

# # Scaling the predicted Price data back to original price scale
# Predictions=TSF.inverse_transform(Predictions_orig)

# # Scaling the y_test Price data back to original price scale
# y_test_orig=TSF.inverse_transform(y_test)

# Scaling the test data back to original scale
#Test_Data=PredictorScalerFit.inverse_transform(X_test)

# TestingData=pd.DataFrame()
# TestingData['Amount']=y_test_orig
# TestingData['PredictedAmountOrig']=Predictions_orig
# TestingData['PredictedAmount']=Predictions
# TestingData.head()

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.metrics import r2_score

# mae = mean_absolute_error(y_true=y_test,y_pred=Predictions_orig)
# #squared True returns MSE value, False returns RMSE value.
# mse = mean_squared_error(y_true=y_test,y_pred=Predictions_orig) #default=True
rmse = mean_squared_error(y_true=y_test,y_pred=Predictions_orig,squared=False)

print("RMSE:",rmse)

In [ ]:
r2_score(y_test, Predictions_orig)